In [ ]:
# stop deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import sklearn
import numpy as np
import pandas as pd
import lime

from lime import lime_tabular
from IPython.display import Image
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

np.random.seed(1)

import warnings
warnings.filterwarnings("ignore")


## Build the model on the generated dataset

In [ ]:
fake_data = pd.read_csv('../data/generated_data_numpy.csv')

original_data = fake_data.copy()
fake_data = fake_data.drop(columns=['Policy_Id', 'Policy_Year', 'Policy_Month'])
fake_data['Accident_Reported'] = np.where(fake_data['Accident_Reported'] == 1, 'Accident', 'No Accident')
fake_data['Accident_Reported'] = fake_data['Accident_Reported'].astype('category')
fake_data['Accident_Reported'] = fake_data['Accident_Reported'].cat.reorder_categories(['Accident', 'No Accident'])


In [ ]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(
    fake_data[[x for x in fake_data.columns if x != 'Accident_Reported']],
    fake_data['Accident_Reported'],
    train_size=0.80)

train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [ ]:
# get a list of categorical features
categorical_features = ['Make', 'Body_Style', 'Model_Color', 'Driver_Hair_Color']

# get a list of numeric features
numeric_features = [x for x in fake_data.columns if x not in categorical_features]


In [ ]:
# get categorical features idx
categorical_features_idx = list(np.where(np.isin(train.columns, categorical_features))[0])

# get numeric features idx
numeric_features_idx = list(np.where(np.isin(train.columns, numeric_features))[0])


In [ ]:
print(categorical_features_idx)

We have to convert text to numbers for LimeExplainer to work unfortunately...

In [ ]:
# Encode categorical features using LabelEncoder
label_encoders = {}
for feature in categorical_features:
    label_encoders[feature] = LabelEncoder()
    train[feature] = label_encoders[feature].fit_transform(train[feature])
    

In [ ]:
# use the label encoders to encode the test data
for feature in categorical_features:
    test[feature] = label_encoders[feature].transform(test[feature])


Let's go ahead and store the label encoded values so that we can have the proper names in the lime explainer.

In [ ]:
categorical_values = {}
for feature in categorical_features:
    # get column number
    col_num = train.columns.get_loc(feature)
    categorical_values[col_num] = list(label_encoders[feature].classes_)

categorical_values


We build a quick pipeline model to use for evaluating the method

In [ ]:
# build a numeric transformer 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())], 
    verbose=True)

# build a categorical transformer with simple imputer and one hot encoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))], 
    verbose=True)

# build a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_idx),
        ('cat', categorical_transformer, categorical_features_idx)])

# build a classifier
clf = RandomForestClassifier(n_estimators=1000, min_samples_split=40)

# build a pipeline to apply the preprocessing and the classifier
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', clf)])


In [ ]:
# fit the pipeline
pipe.fit(train.to_numpy(), labels_train)


In [ ]:
train

In [ ]:
# predictions = pipe.predict(test.to_numpy())
# original_test_data = original_data[original_data.index.isin(test.index)]
# original_test_data.loc[test.index, 'Predictions'] = predictions
# original_test_data.to_csv('../data/predictions.csv', index=False)

#### High-level view ("Earth-view" model statistics)

In [ ]:
# evaluate the pipeline on the test set
roc_auc_score(np.where(labels_test == 'Accident', 1, 0), pipe.predict_proba(test.to_numpy())[:, 0])


In [ ]:
# evaluate accuracy of the pipeline on the test set
pipe.score(test.to_numpy(), labels_test)


## Time for some LIME!

How does LIME actually work?

In [ ]:
Image(filename='../lime_diagram.png')

In [ ]:
# build an explainer for the pipeline
explainer = lime_tabular.LimeTabularExplainer(train.values,
                                              feature_names=train.columns,
                                              categorical_features=categorical_features_idx,
                                              categorical_names=categorical_values,
                                              mode='classification',
                                              class_names=clf.classes_,
                                              discretize_continuous=True,
                                              sample_around_instance=True,
                                              kernel_width=0.0001)


In [ ]:
def peek(x):        
    test_instance = test.iloc[x].copy(deep=True)
    for feature in categorical_features:
        test_instance[feature] = label_encoders[feature].inverse_transform([test_instance[feature]])[0]
    # add labels_test to test_instance
    print(f"Actual Outcome: {labels_test.iloc[x]}")
    return test_instance

test_num = 4
peek(test_num)

In [ ]:
# explain the first instance in the test set
exp = explainer.explain_instance(test.values[test_num], pipe.predict_proba, num_features=7)
exp.show_in_notebook(show_table=True)


In [ ]:
Image(filename='../rvatech_diagram.png')

In [ ]:
# sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

# calculate the "actual" probability based on these features
print(f"Actual: {sigmoid((24939-10000)/5000)}")

# calculate the "global" probability
print(f"Global: {exp.predict_proba[0]}")

# calculate our local interpretable model's probability
print(f"Local: {exp.local_pred[0]}")

Let's find the right value for kernel width!

In [ ]:
abs_values = dict()
range_values = np.arange(0.01, 7.0, 0.1)

for i in range_values:
    explainer = lime_tabular.LimeTabularExplainer(train.values,
                                                feature_names=train.columns,
                                                categorical_features=categorical_features_idx,
                                                categorical_names=categorical_values,
                                                mode='classification',
                                                class_names=clf.classes_,
                                                discretize_continuous=True,
                                                sample_around_instance=True,
                                                kernel_width=i)
    exp = explainer.explain_instance(test.values[test_num], pipe.predict_proba, num_features=7)
    abs_values[i] = abs(exp.local_pred[0] - exp.predict_proba[0])

In [ ]:
# plot abs_values
plt.plot(list(abs_values.keys()), list(abs_values.values()))
# change size of plot
plt.rcParams["figure.figsize"] = (10, 6)
plt.xlabel('Kernel Width')
plt.ylabel('Difference in Probability')
plt.show()

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(train.values,
                                            feature_names=train.columns,
                                            categorical_features=categorical_features_idx,
                                            categorical_names=categorical_values,
                                            mode='classification',
                                            class_names=clf.classes_,
                                            discretize_continuous=True,
                                            sample_around_instance=True,
                                            kernel_width=0.85)
exp = explainer.explain_instance(test.values[test_num], pipe.predict_proba, num_features=7)

In [ ]:
exp = explainer.explain_instance(test.values[test_num], pipe.predict_proba, num_features=7)
exp.show_in_notebook(show_table=True)

In [ ]:
Image(filename='../rvatech_diagram.png')

In [ ]:
exp.as_list()

No strict guarantee of local model approximating global model! In this case, the data-generating process if fairly simple so this worked out.

In [ ]:
perturbed_labels, perturbed_data = explainer._LimeTabularExplainer__data_inverse(test.values[test_num], 100000)
perturbed = pd.DataFrame(perturbed_data, columns=test.columns)
perturbed.head()

Trying another instance.

In [ ]:
test_num = 9
accident_test = test.iloc[test_num].values
peek(test_num)

In [ ]:
# predict the example
reported_prob = lambda x: pipe.predict_proba(x.reshape(1, -1))[0][0]

# verify we get the same probability as above
reported_prob(accident_test)


In [ ]:
exp = explainer.explain_instance(accident_test, pipe.predict_proba, num_features=7)
exp.show_in_notebook(show_table=True, show_all=True)

In [ ]:
Image(filename='../rvatech_diagram.png')

Let's try perturbing this a bit and see how it changes. Based on the above, let's try adjusting a variable to see what happens to predictions.

In [ ]:
temp = accident_test.copy()

impact = {}
for miles in range(0, 20000, 1000):
    temp[4] = miles
    impact[miles] = reported_prob(temp)

pd.DataFrame(impact, index=["Reported Probability"]).T.plot(legend=False, 
                                                            title="Impact of Miles Driven on Reported Probability")
# make chart bigger
plt.gcf().set_size_inches(10, 6)
# add axis labels
plt.xlabel("Miles")
plt.ylabel("Reported Probability")
plt.show()


In [ ]:
impact = {}
for miles in range(0, 20000, 1000):
    temp[4] = miles
    temp[1] = 1
    impact[miles] = reported_prob(temp)

pd.DataFrame(impact, index=["Reported Probability"]).T.plot(legend=False, 
                                                            title="Impact of Miles Driven on Reported Probability")
# make chart bigger
plt.gcf().set_size_inches(10, 6)
# add axis labels
plt.xlabel("Miles")
plt.ylabel("Reported Probability")
plt.show()

In [ ]:
perturbed_labels, perturbed_data = explainer._LimeTabularExplainer__data_inverse(accident_test, 100000)
perturbed = pd.DataFrame(perturbed_data, columns=test.columns)
perturbed.head()

In [ ]:
perturbed = pd.DataFrame(perturbed_data, columns=test.columns)
perturbed_predictions = pipe.predict_proba(perturbed)
perturbed_predictions = perturbed_predictions[:, 0]
perturbed["Risk"] = perturbed_predictions

# highlight dense regions
perturbed.plot.hexbin(x="Miles_Driven", y="Risk", gridsize=20, cmap='Oranges')
# make chart bigger
plt.gcf().set_size_inches(12, 8)
# add line of best fit
# add axis labels
plt.xlabel("Miles")
plt.ylabel("Reported Probability")
plt.plot(np.unique(perturbed["Miles_Driven"]), 
         np.poly1d(np.polyfit(perturbed["Miles_Driven"], 
                              perturbed["Risk"], 1))(np.unique(perturbed["Miles_Driven"])), 
                              color='black')
plt.show()

### Everything below here is just getting explanations for the whole test set

Here, we generate predictions along with our explanations of those predictions.

In [ ]:
predictions = pipe.predict_proba(test)
# # get just the first column
predictions = predictions[:, 0]

In [ ]:
original_test_data = original_data[original_data.index.isin(test.index)]
original_test_data.loc[test.index, 'Predictions'] = predictions
original_test_data

In [ ]:
def explain_instance(row_number=0):
    exp = explainer.explain_instance(test.iloc[row_number], pipe.predict_proba, num_features=7)
    el = exp.as_list()
    # convert el to a dataframe
    el = pd.DataFrame(el, columns=['feature', 'weight'])
    if labels_test.iloc[row_number] == 0:
        el = el.loc[el.weight < 0]
    else:
        el = el.loc[el.weight > 0]
    el["abs_weight"] = el["weight"].abs()
    el = el.sort_values(by='abs_weight', ascending=False)
    el["Policy_Id"] = original_test_data.iloc[row_number]["Policy_Id"]
    return el.head(3)

In [ ]:
# explain all instances and store them in a pandas dataframe
explanations = []
for i in range(0, len(test)):
    explanations.append(explain_instance(i))

explanations = pd.concat(explanations)

In [ ]:
explanations.to_csv("../data/explanations.csv", index=False)